### IMPORT LIBRARY

In [ ]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event,text
import pyodbc
import requests
import inspect
from validate import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from send_email import *

# set to_email
to_email = ['nthieu@savills.com.vn','pthihuongnguyen@savills.com.vn']
# to_email = ['nthieu@savills.com.vn']

### CONNECT TO AZURE SQL

In [ ]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

#Test Connection
try:
    conn = engine.connect()
    result = conn.execute(text("SELECT 1"))
    print("CONNECTION SUCESSFUL!")
except Exception as e:
    print("CONNECTION FAILED:",str(e))

### CONNECT TO SHAREPOINT

In [ ]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

### READ AND LIST FOLDER

In [ ]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

In [69]:
# Lấy danh sách tỉnh
province_list = []
for i in list_folder:
    province_list.append(i.split('/')[7])

# Chọn tỉnh
selected_provinces = []

# Input tỉnh
input_string = input("Nhập tên các tỉnh bạn muốn chọn, cách nhau bằng dấu phẩy (nhập -1 để thoát): ")
input_list = input_string.split(',')

for input_province in input_list:
    input_province = input_province.strip().lower()
    if input_province == "-1":
        break

    matched_provinces = [province for province in province_list if input_province in province.lower()]
    if matched_provinces:
        selected_provinces.extend(matched_provinces)
    else:
        print(f"Không tìm thấy tỉnh chứa từ khóa '{input_province}'.")

# Input Năm và Quý
quarter_year = input("Nhập chuỗi năm và quý (ví dụ: 2023Q1): ")
Year, Quarter = quarter_year[:4], quarter_year[4:]

# In danh sách tỉnh đã chọn, năm và quý
print("Các tỉnh đã chọn:")
for province in selected_provinces:
    print(colored(province,'yellow'))
print(colored("Year:{}, Quarter:{}".format(Year,Quarter),'yellow'))

# Tìm các folder chứa từ khóa
selected_province_folders = [folder for folder in list_folder if any(re.search(keyword, folder, re.IGNORECASE) for keyword in selected_provinces)]
selected_year_quarter = []
for folder in selected_province_folders:
    folder_sub2 = folder + '/' + Year + '/' + Quarter
    selected_year_quarter.append(folder_sub2)
# In danh sách các folder đã tìm thấy
print("Folder các tỉnh cần import:")
for folder in selected_province_folders:
    print(colored(folder,'yellow'))

#Lấy file trong Năm và Quý
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in selected_year_quarter:
    if i.split('/')[7] in selected_provinces and i.split('/')[8] in Year and i.split('/')[9] in Quarter:
        df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_url = df_summ_file['ServerRelativeUrl'].to_list()

#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

Các tỉnh đã chọn:
Thai Binh Province
Year:2023, Quarter:Q1
Folder các tỉnh cần import:
/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Thai Binh Province
Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Thai Binh Province/2023/Q1/IP_Thai Binh_20230331.xlsx


### GET IMPORT FILE

In [70]:
url = []
for i in list_url:
    sector = i.split('/')[-1].split('_')[0].upper()
    if sector not in ('x'):
        url.append(i)
    else:
        pass
url

['/sites/BIHub/Shared Documents/Advisory Data/IP/Flat file/Thai Binh Province/2023/Q1/IP_Thai Binh_20230331.xlsx']

### IMPORT DATA

In [ ]:
'''Prepare ingredients''' 
columns_that_need_unidecode=['Project_Name', 'Sub_Project_Name', 'Developer_Name'
                             , 'City', 'District', 'Target_Industry']
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
#Create multi empty df for tracking autdit step
name_sector = ['IP']
name_sector = [x.lower() for x in name_sector]
for i in name_sector:
    globals()['df_temp_flat_{}'.format(i)] = pd.DataFrame([])
    globals()['df_flat_{}'.format(i)] = pd.DataFrame([])
    globals()['df_new_key_{}'.format(i)] = pd.DataFrame([])  
#-------------------------------------------------------
'''Get data''' 
print('Start Get Data...')
for file_url in tqdm(url):  
    data, file_name, sector = get_data(relative_url, file_url)
    #-------------------------------------------------------
    data = check_date_key(file_url, data)#Check format date_key in flat file   
    data['Project_Name']= np.where(data['Project_Name'].isnull(), data['Sub_Project_Name'], data['Project_Name'])#Fill up project_name if its null
    data['Project_Sub_Type']= np.where(data['Project_Sub_Type'].isnull(), data['Project_Type'], data['Project_Sub_Type'])#Fill up Sub_Type if its null
    
    #Check duplicate sub_name
    print('Start Check duplicate sub_name...')
    data, df_dup = check_duplicate(data, 'Sub_Project_Name')
    if len(df_dup) != 0:
        print(colored('Check duplicate sub_name', 'yellow'))
        df_noti_html = convert_df_to_html(type_html = 1, df = df_dup, type_sector = 2, cnxn = engine)
        run_email(type_sector = 'IP', email_type = 1, user_email = to_email, df_noti_html = df_noti_html)
    else:       
        #-------------------------------------------------------
        
        '''Validation step'''
        print('Start validation step...')
        #Remove unfortmated values
        data = remove_unformated_character(data)
        #Remove unicode characters
        for i in columns_that_need_unidecode:
            data[i] = remove_unicode(data[i])
        #Check dictionary
        print('Start Check dictionary...')
        lst_dict = ['City', 'District', 'Status', 'Sub_Type', 'Type']
        lst_cls = ['City', 'District', 'Status', 'Project_Sub_Type', 'Project_Type']
        for i, j in zip(lst_cls, lst_dict):
            print(f'Start checking dictionary of [{i}:{j}]...')
            data, df_dict = check_dictionary(df_dict, file_name, data, i, j, sector, engine, sp_object)
        print(f'End Check dictionary with df_dict = {len(df_dict)} ...')
        
        if len(df_dict) == 0:
            print(colored('Validate succesfully','green'))
            #-------------------------------------------------------
            '''Import data process'''
            #Check project key
            data = Generate_Additional_Columns(data,url,df_summ_file,BIHub,engine,file_url)
            processed_data, flag_key = check_project_key(file_url, data, sector, engine)
            df_temp_flat_ip = pd.concat([df_temp_flat_ip, data], axis=0)
            df_flat_ip = tracking_flat_file(df_temp_flat_ip, file_url)
            if len(processed_data) != 0:
                df_new_key_ip = check_new_key(df_new_key = df_new_key_ip, processed_data = processed_data, sector = sector)
            #Get key and generate new key (if needed)
            data = get_project_key(flag_key, processed_data, data, sector, engine)
            # insert_to_fresh(file_url, data, cnt_str)
            
            #Test insert_to_fresh
            try:
                result = insert_to_fresh(file_url, data, cnt_str)
                print(colored("insert_to_fresh SUCESSFUL!",'green'))
            except Exception as e:
                print(colored("insert_to_fresh FAILED:",str(e),'red'))
            
        else:
            print(df_dict)
            print(colored('Validate failed','red'))
            pass

### SEND EMAIL

In [ ]:
list_df_flat = [df_flat_ip]
list_df_new_key = [df_new_key_ip]
#Email notify missing in dictionary
if len(df_dict) != 0:
    print(colored('Missing values in dictionary','yellow'))
    df_noti_html = convert_df_to_html(type_html = 2, df = df_dict, cnxn = engine)
    run_email(email_type = 2, user_email = to_email, df_noti_html = df_noti_html)
else:
    pass

#Email notify imported data
df_flat_html, df_query_html = convert_df_to_html(type_html = 3, list_df = list_df_flat, type_sector = 2, cnxn = engine)
run_email(type_sector = 'IP', email_type = 3, user_email = to_email, df_flat_html = df_flat_html, df_query_html = df_query_html)

#Email notify create new key
df_new_key_html = convert_df_to_html(type_html = 4, list_df = list_df_new_key, cnxn = engine)
if len(df_new_key_html) != 0:
    print(colored('Some new keys were created','yellow'))
    run_email(type_sector = 'IP', email_type = 4, user_email = to_email, df_noti_html = df_new_key_html)

In [ ]:
df_dict

### TRACKING AUDIT

In [ ]:
insert_to_tracking(list_df_flat, 'Tracking_IP', engine)

# DEBUG IMPORT

In [71]:
'''Prepare ingredients''' 
columns_that_need_unidecode=['Project_Name', 'Sub_Project_Name', 'Developer_Name'
                             , 'City', 'District', 'Target_Industry']
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
#Create multi empty df for tracking autdit step
name_sector = ['IP']
name_sector = [x.lower() for x in name_sector]
for i in name_sector:

    globals()['df_temp_flat_{}'.format(i)] = pd.DataFrame([])
    globals()['df_flat_{}'.format(i)] = pd.DataFrame([])
    globals()['df_new_key_{}'.format(i)] = pd.DataFrame([])  
#-------------------------------------------------------

In [72]:
'''Get data''' 
print('Start Get Data...')
data, file_name, sector = get_data(relative_url, file_url)
#-------------------------------------------------------
data = check_date_key(file_url, data)#Check format date_key in flat file   
data['Project_Name']= np.where(data['Project_Name'].isnull(), data['Sub_Project_Name'], data['Project_Name'])#Fill up project_name if its null
data['Project_Sub_Type']= np.where(data['Project_Sub_Type'].isnull(), data['Project_Type'], data['Project_Sub_Type'])#Fill up Sub_Type if its null

data.head()

Start Get Data...


,Date_Key,City,District,Project_Name,Sub_Project_Name,Latitude,Longtitude,Phase,Zone,Form,Status,Developer_Name,Developer_Nationality,Project_Type,Project_Sub_Type,LU_Residental,LU_Administration,LU_Green_Water,LU_Transportation,LU_Infra,LU_Commercial,LU_Factory_Building,LU_Warehouse,LU_Other,Target_Industry,Electricity_Capacity(kV),Electricity_Charge(US$/kWh),Water_Capacity(m3/day/night),Water_Charge(US$/m3/mth),Waste_Treatment_Capacity(m3/day/night),Waste_Treatment_Charge(US$/m3/mth),Establish_Year,Launch_Year,Full_Occupied_Year,Expiry_Year,Remain_LUR,Payment_Method,Management_Fee,Management_Fee_Payment_Schedule,Land_Rent_Government,Land_Rent_Government_Payment_Schedule,PROJECT_Land_Area,LAND_Leaseable_Land_Area,LAND_Available_Land_Area,Leasing_Status,Sold_Out_Year,LAND_Leased_Land_Area,Occupancy,LAND_AVG_Rent_Primary_Price,LAND_AVG_Rent_Secondary_Price,LAND_AVG_Rent_Actual,LAND_AVG_Rent_Last_Transaction,RB_Land_Area,RB_Total_GFA,RB_NLA,RB_Leased_Area,RB_AVG_Rent_Primary_Price,RB_AVG_Rent_Secondary_Price,RB_AVG_Rent_Actual,RB_AVG_Rent_Last_Transaction,Grade,Region_Name,Landlord_Type,Flag,Construction_Status
0,20230331,Thai Binh,Quynh Phu,Cau Nghin,Cau Nghin,20.656158,106.439458,NaN,NaN,Present,Active,IDICO,Domestics,Land,Land,-,3.0,30.0,16.00,4.00,-,129.00,5.00,28.0,NaN,110/22KV,0.07000,6500.0,0.500000,5200.0,0.400000,2012.0,0.0,0.0,2062.0,39.0,One-off,0.500000,Annual,10100,Annual,214.00,133.0,118.0,Active,-,15.0,0.110000,80.0,-,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,North,NaN,3,NaN
1,20230331,Thai Binh,Thai Thuy,Lien Ha Thai (Green iP-1),Lien Ha Thai (Green iP-1),20.551725,106.538214,NaN,NaN,Present,Active,Green I-Park,Domestics,Land,Land,0,0.0,0.0,0.00,0.00,0,0.00,0.00,0.0,"High-tech, green-clean technology industries s...",110/22KV - 63 MVA,0.06564,5000.0,0.506543,5000.0,0.422119,2021.0,2022.0,NaN,2071.0,48.0,One-off,0.422119,Annual,Fee of charge within 18 years,0,589.00,353.0,225.0,Active,0,128.0,0.362606,90.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,North,NaN,3,NaN
2,20230331,Thai Binh,Thai Binh city,Song Tra,Song Tra,20.477881,106.324361,NaN,NaN,Present,Active,TBS Song Tra,Domestics,Land,Land,0,0.0,9.3,18.18,3.36,18.52,94.26,6.86,0.0,Multi-purpose industrial park specializing in ...,110/22KV,0.05000,20000.0,0.400000,3700.0,0.300000,2007.0,2009.0,NaN,2057.0,34.0,One-off,0.250000,Annual,10100,Annual,150.48,90.0,6.0,Active,0,84.0,0.933333,85.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,North,NaN,3,NaN
3,20230331,Thai Binh,Tien Hai,Tien Hai,Tien Hai,20.390506,106.523049,NaN,NaN,Present,Active,Viglacera,Domestics,Land,Land,0,0.0,0.0,0.00,0.00,0,0.00,0.00,0.0,- High-tech industries\n- Supporting industrie...,110/22KV - 63 MVA,0.06800,10000.0,0.480000,3500.0,0.420000,2017.0,NaN,NaN,2067.0,44.0,One-off,0.700000,Annual,10100,Annual,446.00,270.0,24.3,Active,0,245.7,0.910000,90.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,North,NaN,3,NaN
4,20230331,Thai Binh,Thai Binh city,Phuc Khanh,Phuc Khanh,20.442028,106.309338,NaN,NaN,Present,Active,Khai Phat Dai Tin,Domestics,Land,Land,0,4.8,18.0,0.00,23.60,0,73.60,0.00,0.0,"Mechanical and electronic industry, agricultur...",NaN,NaN,NaN,NaN,NaN,NaN,2002.0,NaN,NaN,2052.0,29.0,One-off,NaN,NaN,NaN,NaN,120.00,118.0,0.0,Active,0,118.0,1.000000,0.0,0,0,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,North,NaN,3,NaN


In [73]:
#Check duplicate sub_name
print('Start Check duplicate sub_name...')
data, df_dup = check_duplicate(data, 'Sub_Project_Name')

print(f"df_dup = {len(df_dup)}")
if len(df_dup) != 0:
    print(colored('Check duplicate sub_name', 'yellow'))
    df_noti_html = convert_df_to_html(type_html = 1, df = df_dup, type_sector = 2, cnxn = engine)
    run_email(type_sector = 'IP', email_type = 1, user_email = to_email, df_noti_html = df_noti_html)

Start Check duplicate sub_name...
df_dup = 0


In [74]:
'''Validation step'''
print('Start validation step...')
#Remove unfortmated values
data = remove_unformated_character(data)
#Remove unicode characters
for i in columns_that_need_unidecode:
    data[i] = remove_unicode(data[i])
#Check dictionary
print('Start Check dictionary...')
lst_dict = ['City', 'District', 'Status', 'Sub_Type', 'Type']
lst_cls = ['City', 'District', 'Status', 'Project_Sub_Type', 'Project_Type']
for i, j in zip(lst_cls, lst_dict):
    print(f'Start checking dictionary of [{i}:{j}]...')
    data, df_dict = check_dictionary(df_dict, file_name, data, i, j, sector, engine, sp_object)
print(f'End Check dictionary with df_dict = {len(df_dict)} ...')
print(df_dict)

Start validation step...
Start Check dictionary...
Start checking dictionary of [City:City]...
Start checking dictionary of [District:District]...
Start checking dictionary of [Status:Status]...
Start checking dictionary of [Project_Sub_Type:Sub_Type]...
Start checking dictionary of [Project_Type:Type]...
End Check dictionary with df_dict = 0 ...
Empty DataFrame
Columns: [File_Name, Missing_Values, Flag]
Index: []


In [ ]:
parameter = "District"
new_data = "chi linh district"
test_df_dict = pd.read_sql(f"select * from GENERAL.{parameter}_Dictionary where Raw_District = '{new_data}'",engine)
test_df_dict

In [77]:
if len(df_dict) == 0:
    print(colored('Validate succesfully','green'))
    #-------------------------------------------------------
    '''Import data process'''
    #Check project key
    data = Generate_Additional_Columns(data,url,df_summ_file,BIHub,engine,file_url)
    print(data)
    processed_data, flag_key = check_project_key(file_url, data, sector, engine)

    
    df_temp_flat_ip = pd.concat([df_temp_flat_ip, data], axis=0)
    df_flat_ip = tracking_flat_file(df_temp_flat_ip, file_url)
    if len(processed_data) != 0:
        df_new_key_ip = check_new_key(df_new_key = df_new_key_ip, processed_data = processed_data, sector = sector)
    #Get key and generate new key (if needed)
    data = get_project_key(flag_key, processed_data, data, sector, engine)
    # insert_to_fresh(file_url, data, cnt_str)
    
    #Test insert_to_fresh
    try:
        print(colored("Jump-in insert_to_fresh!",'green'))
        # result = insert_to_fresh(file_url, data, cnt_str)
        # print(colored("insert_to_fresh SUCESSFUL!",'green'))
    except Exception as e:
        print(colored("insert_to_fresh FAILED:",str(e),'red'))
    
else:
    print(df_dict)
    print(colored('Validate failed','red'))
    pass

Validate succesfully
    Date_Key                City            District  \
0   20230331  Thai Binh Province  Quynh Phu District   
1   20230331  Thai Binh Province  Thai Thuy District   
2   20230331  Thai Binh Province      Thai Binh City   
3   20230331  Thai Binh Province   Tien Hai District   
4   20230331  Thai Binh Province      Thai Binh City   
5   20230331  Thai Binh Province      Thai Binh City   
6   20230331  Thai Binh Province  Dong Hung District   
7   20230331  Thai Binh Province   Tien Hai District   
8   20230331  Thai Binh Province  Quynh Phu District   
9   20230331  Thai Binh Province  Thai Thuy District   
10  20230331  Thai Binh Province  Thai Thuy District   

                 Project_Name           Sub_Project_Name          Latitude  \
0                   Cau Nghin                  Cau Nghin         20.656158   
1   Lien Ha Thai (Green Ip-1)  Lien Ha Thai (Green Ip-1)         20.551725   
2                    Song Tra                   Song Tra         20.4778